In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import numpy as np 
import pandas as pd


In [ ]:
train = pd.read_csv('train_houses.csv')
numeric_columns = train.select_dtypes(include='number').columns
train = train[numeric_columns]
train = train.drop('Id', axis=1)


In [ ]:
def split_dataset(dataset, test_ratio=0.3):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(train)
train_data = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='SalePrice', task = tfdf.keras.Task.REGRESSION)
valid_data = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label='SalePrice', task = tfdf.keras.Task.REGRESSION)


In [ ]:
rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
rf.fit(x=train_data)

test_data = pd.read_csv('test_houses.csv')
ids = test_data.pop('Id')

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data,task = tfdf.keras.Task.REGRESSION)

sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['SalePrice'] = rf.predict(test_ds)
sample_submission_df.to_csv('sample_submission.csv', index=False)